In [79]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import math
from scipy.spatial.distance import euclidean

matplotlib.use('Qt5Agg')

DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P01_B\\"

SHEET_NAME = ["Segment Position", "Segment Velocity", "Segment Acceleration"]

SIMPLIFIED_JOINTS = ["Head", "Neck", "Left Upper Arm", "Right Upper Arm", "Left Forearm", "Right Forearm",
                     "Left Hand", "Right Hand", "Pelvis", "Left Upper Leg", "Right Upper Leg",
                     "Left Lower Leg", "Right Lower Leg", "Left Foot", "Right Foot"]

SKELETON_CONNECTION_MAP = [["Neck", "Head"],
                         ["Neck", "Left Upper Arm"],
                         ["Left Upper Arm", "Left Forearm"],
                         ["Left Forearm", "Left Hand"],
                         ["Neck", "Right Upper Arm"],
                         ["Right Upper Arm", "Right Forearm"],
                         ["Right Forearm", "Right Hand"],
                         ["Neck", "Pelvis"],
                         ["Pelvis", "Left Upper Leg"],
                         ["Left Upper Leg", "Left Lower Leg"],
                         ["Left Lower Leg", "Left Foot"],
                         ["Pelvis", "Right Upper Leg"],
                         ["Right Upper Leg", "Right Lower Leg"],
                         ["Right Lower Leg", "Right Foot"]]

used_cols = []
for joints in SIMPLIFIED_JOINTS:
    used_cols.append(joints+" x")
    used_cols.append(joints+" y")
    used_cols.append(joints+" z")

DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P01_B\\head10000\\"

SHEET_NAME = ["Segment Position", "Segment Velocity", "Segment Acceleration"]

# motion_recording = pd.read_excel(DATA_PATH + "P01_B_noweightwithgloves.xlsx",
#                          sheet_name = SHEET_NAME,
#                          usecols = used_cols)

position_recording_0 = pd.read_csv(DATA_PATH + "position.csv")[2650:2800]
velocity_recording_0 = pd.read_csv(DATA_PATH + "velocity.csv")[2650:2800]
angular_velocity_recording_0 = pd.read_csv(DATA_PATH + "angular.csv")[2650:2800]
angular_velocity_recording_0 = angular_velocity_recording_0 / 10

In [80]:
def centre(recording):
    columns_with_x = recording.columns[recording.columns.str.contains("x")]
    columns_with_y = recording.columns[recording.columns.str.contains("y")]
    columns_with_z = recording.columns[recording.columns.str.contains("z")]
    recording[columns_with_x] = recording[columns_with_x].sub(recording["Pelvis x"],axis='index')
    recording[columns_with_y] = recording[columns_with_y].sub(recording["Pelvis y"],axis='index')
    recording[columns_with_z] = recording[columns_with_z].sub(recording["Pelvis z"],axis='index')
    return recording

def get_sin_cos(opposite, adjacent):
    hypotenuse = math.sqrt(adjacent ** 2 + opposite ** 2)
    return opposite/hypotenuse, adjacent/hypotenuse

def rotate(frame, sin, cos):
    rotation_matrix = np.array([[cos, sin, 0],[-sin, cos, 0],[0, 0, 1]])
    result_array = []
    for joint in range(0, len(frame), 3):
        joint_vector = np.array([[frame.iloc[joint]], [frame.iloc[joint+1]], [frame.iloc[joint+2]]])
        result_array = np.append(result_array, np.dot(rotation_matrix, joint_vector).T)
    return pd.Series(result_array)

def confront(recording):
    sin_cos = pd.DataFrame()
    sin_cos[["sin","cos"]] = recording.apply(lambda x: get_sin_cos(x["Left Upper Arm y"] - x["Right Upper Arm y"], x["Left Upper Arm x"] - x["Right Upper Arm x"]), axis=1, result_type="expand")
    recording[recording.columns] = recording.apply(lambda x: rotate(x, sin_cos["sin"][x.name], sin_cos["cos"][x.name]), axis=1, result_type="expand")
    return recording


In [81]:
body_size = {}
for joint_connection in SKELETON_CONNECTION_MAP:
    endpoint_0 = joint_connection[0]
    endpoint_1 = joint_connection[1]
    body_size[(endpoint_0, endpoint_1)] = euclidean(velocity_recording_0[[endpoint_0+" x", endpoint_0+" y", endpoint_0+" z"]].iloc[0], velocity_recording_0[[endpoint_1+" x", endpoint_1+" y", endpoint_1+" z"]].iloc[0])

angular_velocity = pd.DataFrame()
for joint_connection in SKELETON_CONNECTION_MAP:
    endpoint_0 = joint_connection[0]
    endpoint_1 = joint_connection[1]
    angular_velocity[(endpoint_1, endpoint_0, "x")] = (velocity_recording_0[endpoint_1+" x"] - velocity_recording_0[endpoint_0+" x"])
    angular_velocity[(endpoint_1, endpoint_0, "y")] = (velocity_recording_0[endpoint_1+" y"] - velocity_recording_0[endpoint_0+" y"])
    angular_velocity[(endpoint_1, endpoint_0, "z")] = (velocity_recording_0[endpoint_1+" z"] - velocity_recording_0[endpoint_0+" z"])


In [82]:
def update(index, position_recording, skeleton, velocity_recording, visualized_velocities):
    position = position_recording.iloc[index]
    velocity = velocity_recording.iloc[index]
    for line, joint_velocity, joint_connection in zip(skeleton, visualized_velocities, SKELETON_CONNECTION_MAP):
        endpoint_0 = joint_connection[0]
        endpoint_1 = joint_connection[1]

        line_x = [position[endpoint_0 + " x"], position[endpoint_1 + " x"]]
        line_y = [position[endpoint_0 + " y"], position[endpoint_1 + " y"]]
        line_z = [position[endpoint_0 + " z"], position[endpoint_1 + " z"]]

        # joint_velocity_x = [position[endpoint_1 + " x"], position[endpoint_1 + " x"] - velocity[(endpoint_1, endpoint_0,"x")]]
        # joint_velocity_y = [position[endpoint_1 + " y"], position[endpoint_1 + " y"] - velocity[(endpoint_1, endpoint_0,"y")]]
        # joint_velocity_z = [position[endpoint_1 + " z"], position[endpoint_1 + " z"] - velocity[(endpoint_1, endpoint_0,"z")]]

        joint_velocity_x = [position[endpoint_1 + " x"], position[endpoint_1 + " x"] + velocity[endpoint_1 + " x"]]
        joint_velocity_y = [position[endpoint_1 + " y"], position[endpoint_1 + " y"] + velocity[endpoint_1 + " y"]]
        joint_velocity_z = [position[endpoint_1 + " z"], position[endpoint_1 + " z"] + velocity[endpoint_1 + " z"]]

        line.set_data([line_x, line_y])
        line.set_3d_properties(line_z)

        joint_velocity.set_data([joint_velocity_x, joint_velocity_y])
        joint_velocity.set_3d_properties(joint_velocity_z)

frames_number = 150

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.set(xlim3d=(-1, 1), xlabel='X')
ax.set(ylim3d=(-1, 1), ylabel='Y')
ax.set(zlim3d=(-1, 1), zlabel='Z')

# skeleton_0 = [ax.plot([], [], [], c="r")[0] for _ in SKELETON_CONNECTION_MAP]
# skeleton_1 = [ax.plot([], [], [], c="b")[0] for _ in SKELETON_CONNECTION_MAP]
skeleton_2 = [ax.plot([], [], [], c="g")[0] for _ in SKELETON_CONNECTION_MAP]
visualized_velocities = [ax.plot([], [], [], c="r")[0] for _ in SKELETON_CONNECTION_MAP]

# position_recording_0_copy = position_recording_0.copy()
# position_recording_0_centred_copy = centre(position_recording_0).copy()
# position_recording_0_centred_confronted = confront(centre(position_recording_0))

# ani_0 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True, fargs=(position_recording_0_copy,skeleton_0))
# ani_1 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True, fargs=(position_recording_0_centred_copy,skeleton_1))
ani_2 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True,
                                       fargs=(position_recording_0, skeleton_2, angular_velocity_recording_0, visualized_velocities))

is_paused = False
def toggle_pause(anis):
    global is_paused
    if is_paused:
        for ani in anis:
            ani.resume()
    else:
        for ani in anis:
            ani.pause()
    is_paused = not is_paused


fig.canvas.mpl_connect('button_press_event', lambda event: toggle_pause([ani_2]))

plt.show()